In [98]:
import torch
import torch.nn as nn
import numpy as np

In [99]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load data

In [100]:
x1, x2, y = np.loadtxt('non_linearly_separable.txt', skiprows=1, unpack=True)

In [101]:
def one_hot_encode(Y):
    n_labels = Y.shape[0]
    result = np.zeros((n_labels, 2))
    for i in range(n_labels):
        result[i][Y[i]] = 1
    return result

In [102]:
x = np.column_stack((x1, x2))

X_train = torch.from_numpy(x.astype(np.float32))

Y_train_unencoded = y.astype(int).reshape(-1, 1)
Y_train_ohe = one_hot_encode(Y_train_unencoded)
Y_train = torch.from_numpy(Y_train_ohe.astype(np.float32)).view(-1,2)

X_test = torch.from_numpy(x.astype(np.float32))
Y_test = torch.from_numpy(y.astype(np.float32)).view(-1,1)


![title](img/picture.png)

## Neural Network 9-1 (BCELoss)

In [103]:
class OnehiddenNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(OnehiddenNN, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size) 
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(hidden_size, num_classes)  
        self.softmax = nn.Softmax()
    def forward(self, x):
        out = self.linear1(x)
        out = self.sigmoid(out)
        out = self.linear2(out)
        y_pred = self.softmax(out) 
        return y_pred

In [104]:
# setting model 的設定、initial model的參數、建立model
n_iters = 800
learning_rate = 0.1
n_hidden_nodes=10
criterion = nn.BCELoss()
input_size = X_train.shape[1]
hidden_size = n_hidden_nodes
num_classes= Y_train.shape[1]

model = OnehiddenNN(input_size, hidden_size ,num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [105]:
# for 迴圈跑數值
for iters in range(n_iters):

    y_pred = model(X_train)
    loss = criterion(y_pred, Y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (iters+1) % 50 == 0:
        print(f'Iteration: {iters+1}, Loss = {loss.item():.4f}') 

/var/folders/9_/gr09sxx135g1s84jn44jfld80000gn/T/ipykernel_91649/3001672713.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = self.softmax(out)


Iteration: 50, Loss = 0.5638
Iteration: 100, Loss = 0.4571
Iteration: 150, Loss = 0.2382
Iteration: 200, Loss = 0.1921
Iteration: 250, Loss = 0.1591
Iteration: 300, Loss = 0.1354
Iteration: 350, Loss = 0.1182
Iteration: 400, Loss = 0.1023
Iteration: 450, Loss = 0.0902
Iteration: 500, Loss = 0.0807
Iteration: 550, Loss = 0.0728
Iteration: 600, Loss = 0.0661
Iteration: 650, Loss = 0.0603
Iteration: 700, Loss = 0.0553
Iteration: 750, Loss = 0.0510
Iteration: 800, Loss = 0.0470


In [106]:
with torch.no_grad():
    y_predicted = model(X_test)
    _,y_predicted_cls = torch.max(y_predicted,1)
    acc = (y_predicted_cls.eq(torch.squeeze(Y_test)).sum() / Y_test.shape[0] )*100
    print(f'accuracy: {acc.item():.4f}%')

accuracy: 98.0000%


/var/folders/9_/gr09sxx135g1s84jn44jfld80000gn/T/ipykernel_91649/3001672713.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = self.softmax(out)


## Neural Network 9-2 (CrossEntropyLoss)

In [107]:
class OnehiddenNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(OnehiddenNN, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size) 
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(hidden_size, num_classes)  
    def forward(self, x):
        out = self.linear1(x)
        out = self.sigmoid(out)
        y_pred = self.linear2(out)
        return y_pred

In [108]:
n_iters = 800
learning_rate = 0.1
n_hidden_nodes=10
criterion = nn.CrossEntropyLoss()
input_size = X_train.shape[1]
hidden_size = n_hidden_nodes
num_classes= Y_train.shape[1]

model = OnehiddenNN(input_size, hidden_size ,num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [109]:
for iters in range(n_iters):

    y_pred = model(X_train)
    loss = criterion(y_pred, Y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (iters+1) % 50 == 0:
        print(f'Iteration: {iters+1}, Loss = {loss.item():.4f}') 

Iteration: 50, Loss = 0.5781
Iteration: 100, Loss = 0.4735
Iteration: 150, Loss = 0.2378
Iteration: 200, Loss = 0.1965
Iteration: 250, Loss = 0.1691
Iteration: 300, Loss = 0.1521
Iteration: 350, Loss = 0.1371
Iteration: 400, Loss = 0.1146
Iteration: 450, Loss = 0.0959
Iteration: 500, Loss = 0.0859
Iteration: 550, Loss = 0.0795
Iteration: 600, Loss = 0.0746
Iteration: 650, Loss = 0.0706
Iteration: 700, Loss = 0.0672
Iteration: 750, Loss = 0.0642
Iteration: 800, Loss = 0.0602


In [110]:
with torch.no_grad():
    y_predicted = model(X_test)
    _,y_predicted_cls = torch.max(y_predicted,1)
    acc = (y_predicted_cls.eq(torch.squeeze(Y_test)).sum() / Y_test.shape[0] )*100
    print(f'accuracy: {acc.item():.4f}%')

accuracy: 96.3333%
